In [6]:
import csv
import pandas as pd
import re

In [7]:
# Data from https://www.nporadio2.nl/top2000
df = pd.read_csv("../source/nporadio2-Top-2000-2023.csv")

In [8]:
remove_suffixes = [" (Albumversie)", " (Live)"]

df['cleaned_title'] = df['titel']

for suffix in remove_suffixes:
	df['cleaned_title'] = df['cleaned_title'].str.replace(suffix, '')

def words(title, with_parends):
	'''
	words('Street Spirit (Fade Out)', True) -> ['Street', 'Spirit', 'Fade', 'Out']
	words('Street Spirit (Fade Out)', False) -> ['Street', 'Spirit']
	'''
	title = title.strip()
	if not with_parends:
		title = re.sub(r'\([^)]*\)', '', title).strip()
	
	return [word.strip("()") for word in re.split(r'\s+', title)]

df['starts_with'] = df['cleaned_title'].apply(lambda x: words(x, False)[0].lower())
df['ends_with'] = df['cleaned_title'].apply(lambda x: words(x, False)[-1].lower())

df['starts_with_inc_parends'] = df['cleaned_title'].apply(lambda x: words(x, True)[0].lower())
df['ends_with_inc_parends'] = df['cleaned_title'].apply(lambda x: words(x, True)[-1].lower())

with open("top2000-labels.txt", "w") as f:
	f.write("\n".join(df['cleaned_title']))

labelled_edges = []

for i, src in df.iterrows():
	linking_words = set([src['ends_with'], src['ends_with_inc_parends']])
	targets = df[df['starts_with'].isin(linking_words) | df['starts_with_inc_parends'].isin(linking_words)]
	for j, tar in targets.iterrows():
		if src['cleaned_title'] == tar['cleaned_title']:
			continue
		labelled_edges.append(((i, src['cleaned_title']), (j, tar['cleaned_title'])))

with open("top2000-graph.txt", "w") as f:
	f.write(str(len(df['cleaned_title'])) + "\n")

	for (i, _), (j, _) in labelled_edges:
		f.write(f'{i} {j}\n')

with open("top2000-labelled-edges.txt", "w") as f:
	for (i, src_title), (j, tar_title) in labelled_edges:
		f.write(f"{src_title} -> {tar_title} ({i} -> {j})\n")

In [12]:
import subprocess

with open("top2000-graph.txt", "r") as f:
	result = subprocess.run(['../../lpath','BRUTE_FORCE'], stdin=f, stdout=subprocess.PIPE, text=True)

path = list(map(int, re.search(r'Longest path: (.+)\s', result.stdout).group(1).split()))
titles = [df.iloc[i]['cleaned_title'] for i in path]

print(result.stdout)
print('\n'.join(titles))

Read graph. vertices: 2000, edges: 3465
Search mode: BRUTE_FORCE
Longest path length: 30
Longest path: 114 1644 458 1359 1129 556 1831 1994 1045 1661 1344 1682 879 1040 670 100 77 1813 1915 880 1372 250 1819 1747 1079 1837 208 1922 120 814 1712 
Time: 1.049675s

Sign Of The Times
Times Were When
When Doves Cry
Cry Baby
Baby Can I Hold You
You Shook Me All Night Long
Long Cool Woman (In A Black Dress)
Woman
Woman In Love
Love Is All
All You Need Is Love
Love Me Just A Little Bit More (Totally Hooked On You)
You Need To Calm Down
Down Down
Down Under
Under The Bridge
Bridge Over Troubled Water
Water Of Love
Love Really Hurts Without You
You
You Can't Hurry Love
Love You More
More Than This
This Charming Man
Man On The Moon
Moon Over Bourbon Street
Street Spirit (Fade Out)
Out In The Fields
Fields Of Gold
Gold
Gold On The Ceiling


In [13]:
import subprocess

with open("top2000-graph.txt", "r") as f:
	result = subprocess.run(['../../lpath','DFBNB'], stdin=f, stdout=subprocess.PIPE, text=True)

print(result.stdout)


# Depth first Branch and Bound takes longer

Read graph. vertices: 2000, edges: 3465
Search mode: DFBNB
Longest path length: 30
Longest path: 114 1644 458 1359 1129 556 1831 1994 1045 1661 1344 1682 879 1040 670 100 77 1813 1915 880 1372 250 1819 1747 1079 1837 208 1922 120 814 1712 
Time: 6.971052s

